In [2]:
import math
import numpy as np
import pandas as pd
import os
import glob
import pickle
import xgboost as xgb
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import f_regression
from sklearn.feature_selection import mutual_info_regression

In [3]:
def bestfeatureslist (dfX, listbestvalues, wantedfeatures):
    output = pd.DataFrame(data=[])
    for i in range (0, wantedfeatures):
        output = pd.concat([output, dfX.iloc[:, listbestvalues[i]]], axis = 1).reindex(dfX.index)
    return output

In [4]:
def split_data_labels_interview (csv_file):
    df = pd.read_csv(csv_file)
    x_array = df.iloc[:,1:-6]
    y_array = df.iloc[:,230:]
    return x_array, y_array

In [5]:
def split_data_labels_openness (csv_file):
    df = pd.read_csv(csv_file)
    x_array = df.iloc[:,1:-6]
    y_array = df.iloc[:,225]
    return x_array, y_array

In [5]:
def split_data_labels_conscientiousness (csv_file):
    df = pd.read_csv(csv_file)
    x_array = df.iloc[:,1:-6]
    y_array = df.iloc[:,226]
    return x_array, y_array

In [6]:
def split_data_labels_extroversion (csv_file):
    df = pd.read_csv(csv_file)
    x_array = df.iloc[:,1:-6]
    y_array = df.iloc[:,227]
    return x_array, y_array

In [7]:
def split_data_labels_agreeableness (csv_file):
    df = pd.read_csv(csv_file)
    x_array = df.iloc[:,1:-6]
    y_array = df.iloc[:,228]
    return x_array, y_array

In [8]:
def split_data_labels_neuroticism (csv_file):
    df = pd.read_csv(csv_file)
    x_array = df.iloc[:,1:-6]
    y_array = df.iloc[:,229]
    return x_array, y_array

In [11]:
listbestvalues_interview = [130, 212, 178, 42, 74, 25, 8, 171, 184, 72, 31, 180, 172, 61, 73, 177, 223, 192, 125, 110, 142, 213, 181, 19, 100, 76, 27, 36, 163, 7, 187, 59, 126, 109, 111, 123, 41, 158, 115, 120, 71, 79, 11, 15, 208, 118, 102, 75, 112, 176, 195, 209, 121, 105, 222, 136, 220, 16, 124, 80, 108, 191, 30, 54, 185, 168, 210, 37, 52, 117, 201, 221, 205, 190, 81, 99, 60, 182, 38, 198, 131, 165, 207, 197, 33, 173, 83, 43, 29, 186, 32, 68, 66, 129, 216, 96, 103, 24, 206, 89, 174, 2, 166, 98, 22, 132, 65, 62, 3, 95, 157, 57, 6, 17, 67, 70, 69, 85, 107, 134, 23, 188, 116, 194, 13, 153, 82, 78, 56, 86, 156, 106, 119, 28, 55, 84, 128, 77, 179, 21, 164, 218, 170, 147, 211, 135, 154, 47, 150, 183, 141, 155, 138, 219, 160, 215, 26, 101, 104, 97, 93, 94, 203, 9, 151, 196, 58, 50, 167, 146, 87, 53, 4, 45, 35, 51, 127, 175, 159, 149, 10, 169, 193, 63, 162, 202, 92, 44, 204, 217, 214, 199, 200, 49, 5, 189, 48, 64, 46, 40, 39, 88, 90, 91, 34, 1, 113, 114, 122, 133, 137, 139, 140, 143, 144, 145, 148, 152, 20, 161, 18, 14, 12, 0]

In [8]:
listbestvalues_openness = [42, 36, 19, 37, 25, 51, 8, 48, 27, 6, 53, 61, 33, 41, 39, 130, 38, 198, 186, 71, 16, 101, 141, 223, 192, 21, 83, 106, 12, 92, 188, 150, 30, 189, 182, 120, 118, 68, 70, 17, 104, 3, 52, 11, 127, 179, 205, 108, 49, 176, 32, 187, 115, 199, 184, 58, 123, 133, 102, 0, 109, 103, 5, 107, 56, 135, 168, 197, 29, 116, 147, 105, 77, 117, 124, 85, 216, 119, 34, 169, 212, 132, 46, 173, 91, 202, 63, 137, 72, 64, 13, 9, 163, 185, 23, 162, 43, 181, 177, 121, 35, 180, 113, 90, 167, 40, 59, 151, 4, 213, 191, 154, 175, 131, 138, 222, 129, 142, 28, 18, 62, 60, 84, 20, 194, 125, 164, 155, 75, 7, 178, 143, 156, 31, 153, 171, 174, 15, 136, 14, 161, 140, 86, 10, 114, 112, 166, 217, 110, 128, 211, 93, 66, 159, 195, 201, 200, 55, 73, 74, 196, 221, 203, 126, 193, 1, 190, 76, 111, 69, 2, 204, 220, 206, 78, 207, 208, 209, 210, 57, 67, 214, 215, 65, 218, 219, 146, 79, 97, 44, 139, 50, 24, 160, 158, 157, 98, 144, 94, 145, 152, 95, 96, 149, 148, 165, 89, 88, 54, 183, 80, 81, 82, 45, 26, 22, 87, 134, 47, 100, 172, 99, 170, 122]

In [10]:
listbestvalues_conscientiousness = [150, 221, 82, 169, 57, 180, 173, 77, 129, 145, 36, 55, 93, 185, 88, 30, 204, 119, 209, 53, 167, 83, 87, 137, 29, 168, 179, 59, 203, 170, 66, 151, 115, 160, 177, 90, 108, 74, 136, 159, 68, 175, 118, 98, 158, 84, 222, 110, 117, 72, 128, 61, 178, 141, 60, 132, 8, 86, 50, 210, 78, 152, 155, 164, 149, 182, 156, 120, 154, 166, 218, 69, 200, 92, 219, 163, 109, 127, 201, 161, 97, 126, 206, 144, 153, 102, 212, 23, 162, 111, 99, 140, 125, 10, 100, 186, 114, 63, 143, 202, 95, 208, 105, 205, 184, 171, 89, 76, 73, 94, 148, 62, 124, 75, 5, 147, 133, 103, 181, 67, 1, 70, 12, 195, 54, 81, 142, 106, 96, 131, 116, 134, 107, 191, 174, 176, 85, 64, 165, 71, 123, 45, 130, 138, 26, 183, 157, 146, 19, 80, 192, 104, 211, 196, 139, 9, 27, 52, 56, 58, 32, 47, 198, 31, 112, 20, 18, 113, 187, 46, 193, 22, 2, 213, 51, 189, 79, 13, 188, 220, 16, 223, 38, 15, 214, 33, 91, 135, 11, 28, 14, 44, 199, 0, 3, 172, 49, 216, 194, 101, 48, 6, 40, 121, 41, 122, 4, 7, 217, 215, 65, 21, 17, 197, 24, 25, 207, 190, 34, 43, 37, 39, 42, 35]

In [11]:
listbestvalues_extroversion = [42, 25, 38, 8, 198, 41, 31, 46, 197, 191, 19, 6, 143, 2, 3, 125, 63, 202, 4, 20, 37, 200, 16, 192, 99, 100, 220, 194, 27, 32, 127, 51, 159, 109, 36, 112, 212, 172, 0, 130, 108, 175, 5, 56, 173, 74, 7, 30, 217, 105, 209, 23, 75, 188, 1, 134, 94, 67, 47, 111, 213, 184, 21, 107, 174, 208, 70, 15, 89, 48, 114, 44, 52, 196, 170, 133, 195, 113, 90, 142, 59, 149, 84, 215, 18, 165, 210, 160, 176, 123, 139, 40, 144, 91, 28, 141, 98, 155, 124, 140, 121, 145, 33, 190, 216, 116, 43, 150, 218, 10, 110, 88, 203, 169, 183, 34, 13, 29, 62, 164, 66, 49, 206, 132, 78, 151, 97, 119, 95, 128, 214, 76, 58, 177, 161, 199, 55, 69, 156, 83, 45, 148, 24, 65, 14, 87, 96, 166, 71, 11, 146, 135, 181, 179, 178, 180, 39, 182, 185, 186, 187, 189, 193, 201, 9, 204, 205, 207, 211, 219, 221, 222, 12, 131, 17, 77, 101, 26, 93, 92, 86, 85, 82, 81, 80, 79, 73, 171, 72, 68, 64, 61, 60, 57, 54, 53, 50, 35, 102, 103, 104, 106, 168, 167, 163, 162, 158, 157, 154, 153, 152, 147, 138, 137, 136, 129, 22, 126, 122, 120, 118, 117, 115, 223]

In [12]:
listbestvalues_agreeableness = [150, 186, 166, 163, 147, 8, 168, 164, 170, 141, 64, 145, 212, 47, 27, 122, 165, 108, 94, 162, 182, 207, 172, 54, 220, 144, 116, 205, 91, 55, 83, 167, 213, 41, 185, 159, 149, 181, 179, 133, 171, 148, 173, 67, 96, 31, 33, 151, 61, 180, 152, 84, 131, 124, 49, 28, 222, 80, 32, 93, 174, 104, 35, 126, 143, 42, 43, 161, 175, 183, 121, 204, 184, 221, 71, 92, 214, 58, 29, 119, 187, 98, 100, 140, 37, 87, 36, 52, 95, 153, 169, 63, 17, 110, 177, 88, 45, 23, 139, 59, 68, 103, 2, 97, 196, 73, 102, 62, 191, 105, 158, 190, 200, 137, 146, 127, 176, 89, 13, 154, 34, 77, 65, 132, 125, 115, 44, 223, 81, 99, 192, 85, 24, 202, 160, 53, 22, 155, 217, 56, 38, 210, 142, 156, 113, 50, 112, 106, 60, 66, 117, 118, 20, 21, 4, 130, 3, 19, 74, 70, 178, 90, 76, 12, 198, 30, 82, 25, 129, 72, 203, 208, 6, 75, 114, 128, 197, 78, 216, 199, 18, 5, 219, 218, 201, 16, 15, 14, 206, 215, 7, 209, 9, 10, 211, 11, 138, 39, 195, 194, 135, 134, 123, 120, 157, 1, 111, 109, 107, 101, 86, 79, 69, 57, 51, 48, 46, 40, 136, 26, 188, 189, 193, 0]

In [13]:
listbestvalues_neuroticism = [42, 8, 41, 36, 197, 25, 198, 132, 155, 16, 21, 192, 7, 212, 142, 218, 222, 6, 53, 30, 18, 144, 4, 133, 217, 19, 27, 214, 179, 55, 74, 143, 193, 185, 129, 203, 154, 23, 32, 200, 3, 147, 80, 189, 59, 49, 88, 186, 219, 76, 172, 151, 140, 148, 77, 209, 20, 92, 101, 130, 208, 54, 61, 82, 191, 171, 78, 9, 141, 45, 165, 81, 38, 205, 11, 187, 12, 87, 91, 176, 106, 207, 194, 95, 161, 206, 17, 195, 199, 46, 139, 196, 52, 28, 153, 105, 83, 168, 167, 184, 58, 156, 104, 47, 98, 158, 145, 97, 152, 138, 14, 163, 150, 146, 57, 96, 215, 136, 210, 34, 211, 64, 40, 108, 178, 86, 66, 166, 137, 202, 10, 123, 33, 79, 0, 110, 164, 67, 94, 174, 13, 221, 175, 126, 103, 182, 119, 99, 125, 102, 115, 69, 201, 31, 181, 180, 204, 213, 183, 190, 216, 170, 169, 220, 173, 188, 177, 112, 162, 160, 73, 72, 71, 70, 68, 65, 63, 62, 60, 56, 51, 50, 48, 44, 43, 39, 37, 35, 29, 26, 24, 22, 15, 5, 2, 75, 84, 85, 120, 159, 157, 149, 135, 134, 131, 128, 127, 124, 122, 121, 118, 89, 117, 116, 114, 113, 1, 111, 109, 107, 100, 93, 90, 223]

In [6]:
csv_file = "complete_CSV_Train.csv"
x_train, y_train = split_data_labels_openness(csv_file)
csv_file = "complete_CSV_validation.csv"
x_test, y_test = split_data_labels_openness(csv_file)

In [17]:
x_train_10 = bestfeatureslist(x_train, listbestvalues_interview, 17
                             )
x_test_10 = bestfeatureslist(x_test, listbestvalues_openness, 10)

In [59]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

data = pd.read_csv("complete_CSV_Train.csv")

X, y = x_train_10, y_train #1
    
params = { 'max_depth': [3,6,10],
           'learning_rate': [0.01, 0.05, 0.1],
           'n_estimators': [100, 500, 1000],
           'colsample_bytree': [0.3, 0.7]}

xgbr = xgb.XGBRegressor(seed = 20)
clf = GridSearchCV(estimator=xgbr, 
                   param_grid=params,
                   scoring='neg_mean_absolute_error', 
                   verbose=1)

clf.fit(X, y)
print("Best parameters:", clf.best_params_)
print("Lowest mae: ", (-clf.best_score_)**(1/2.0))

Fitting 5 folds for each of 54 candidates, totalling 270 fits
Best parameters: {'colsample_bytree': 0.7, 'learning_rate': 0.01, 'max_depth': 6, 'n_estimators': 500}
Lowest mae:  0.33945826336344803


In [16]:
x_train_50 = bestfeatureslist(x_train, listbestvalues_openness, 50)
x_test_50 = bestfeatureslist(x_test, listbestvalues_openness, 50)

In [17]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

data = pd.read_csv("complete_CSV_Train.csv")

X, y = x_train_50, y_train #1
    
params = { 'max_depth': [3,6,10],
           'learning_rate': [0.01, 0.05, 0.1],
           'n_estimators': [100, 500, 1000],
           'colsample_bytree': [0.3, 0.7]}

xgbr = xgb.XGBRegressor(seed = 20)
clf = GridSearchCV(estimator=xgbr, 
                   param_grid=params,
                   scoring='neg_mean_absolute_error', 
                   verbose=1)

clf.fit(X, y)
print("Best parameters:", clf.best_params_)
print("Lowest mae: ", (-clf.best_score_)**(1/2.0))

Fitting 5 folds for each of 54 candidates, totalling 270 fits
Best parameters: {'colsample_bytree': 0.7, 'learning_rate': 0.01, 'max_depth': 10, 'n_estimators': 1000}
Lowest mae:  0.3202892847666448


In [18]:
x_train_100 = bestfeatureslist(x_train, listbestvalues_openness, 100)
x_test_100 = bestfeatureslist(x_test, listbestvalues_openness, 100)

In [19]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

data = pd.read_csv("complete_CSV_Train.csv")

X, y = x_train_100, y_train #1
    
params = { 'max_depth': [3,6,10],
           'learning_rate': [0.01, 0.05, 0.1],
           'n_estimators': [100, 500, 1000],
           'colsample_bytree': [0.3, 0.7]}

xgbr = xgb.XGBRegressor(seed = 20)
clf = GridSearchCV(estimator=xgbr, 
                   param_grid=params,
                   scoring='neg_mean_absolute_error', 
                   verbose=1)

clf.fit(X, y)
print("Best parameters:", clf.best_params_)
print("Lowest mae: ", (-clf.best_score_)**(1/2.0))

Fitting 5 folds for each of 54 candidates, totalling 270 fits
Best parameters: {'colsample_bytree': 0.7, 'learning_rate': 0.01, 'max_depth': 10, 'n_estimators': 1000}
Lowest mae:  0.3175640735059191


In [20]:
x_train = bestfeatureslist(x_train, listbestvalues_openness, 224)
x_test = bestfeatureslist(x_test, listbestvalues_openness, 224)

In [21]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

data = pd.read_csv("complete_CSV_Train.csv")

X, y = x_train, y_train #1
    
params = { 'max_depth': [3,6,10],
           'learning_rate': [0.01, 0.05, 0.1],
           'n_estimators': [100, 500, 1000],
           'colsample_bytree': [0.3, 0.7]}

xgbr = xgb.XGBRegressor(seed = 20)
clf = GridSearchCV(estimator=xgbr, 
                   param_grid=params,
                   scoring='neg_mean_absolute_error', 
                   verbose=1)

clf.fit(X, y)
print("Best parameters:", clf.best_params_)
print("Lowest mae: ", (-clf.best_score_)**(1/2.0))

Fitting 5 folds for each of 54 candidates, totalling 270 fits
Best parameters: {'colsample_bytree': 0.3, 'learning_rate': 0.01, 'max_depth': 6, 'n_estimators': 1000}
Lowest mae:  0.3146695120795427


In [15]:
csv_file = "complete_CSV_Train.csv"
x_train, y_train = split_data_labels_conscientiousness(csv_file)
csv_file = "complete_CSV_validation.csv"
x_test, y_test = split_data_labels_conscientiousness(csv_file)

In [23]:
x_train_10 = bestfeatureslist(x_train, listbestvalues_conscientiousness, 10)
x_test_10 = bestfeatureslist(x_test, listbestvalues_conscientiousness, 10)

In [24]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

data = pd.read_csv("complete_CSV_Train.csv")

X, y = x_train_10, y_train #1
    
params = { 'max_depth': [3,6,10],
           'learning_rate': [0.01, 0.05, 0.1],
           'n_estimators': [100, 500, 1000],
           'colsample_bytree': [0.3, 0.7]}

xgbr = xgb.XGBRegressor(seed = 20)
clf = GridSearchCV(estimator=xgbr, 
                   param_grid=params,
                   scoring='neg_mean_absolute_error', 
                   verbose=1)

clf.fit(X, y)
print("Best parameters:", clf.best_params_)
print("Lowest mae: ", (-clf.best_score_)**(1/2.0))

Fitting 5 folds for each of 54 candidates, totalling 270 fits
Best parameters: {'colsample_bytree': 0.3, 'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 1000}
Lowest mae:  0.3484722705490323


In [18]:
x_train_50 = bestfeatureslist(x_train, listbestvalues_conscientiousness, 50)
x_test_50 = bestfeatureslist(x_test, listbestvalues_conscientiousness, 50)

In [19]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

data = pd.read_csv("complete_CSV_Train.csv")

X, y = x_train_50, y_train #1
    
params = { 'max_depth': [3,6,10],
           'learning_rate': [0.01, 0.05, 0.1],
           'n_estimators': [100, 500, 1000],
           'colsample_bytree': [0.3, 0.7]}

xgbr = xgb.XGBRegressor(seed = 20)
clf = GridSearchCV(estimator=xgbr, 
                   param_grid=params,
                   scoring='neg_mean_absolute_error', 
                   verbose=1)

clf.fit(X, y)
print("Best parameters:", clf.best_params_)
print("Lowest mae: ", (-clf.best_score_)**(1/2.0))

Fitting 5 folds for each of 54 candidates, totalling 270 fits
Best parameters: {'colsample_bytree': 0.7, 'learning_rate': 0.01, 'max_depth': 6, 'n_estimators': 500}
Lowest mae:  0.34424855720129777


In [20]:
x_train_100 = bestfeatureslist(x_train, listbestvalues_conscientiousness, 100)
x_test_100 = bestfeatureslist(x_test, listbestvalues_openness, 100)

In [21]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

data = pd.read_csv("complete_CSV_Train.csv")

X, y = x_train_100, y_train #1
    
params = { 'max_depth': [3,6,10],
           'learning_rate': [0.01, 0.05, 0.1],
           'n_estimators': [100, 500, 1000],
           'colsample_bytree': [0.3, 0.7]}

xgbr = xgb.XGBRegressor(seed = 20)
clf = GridSearchCV(estimator=xgbr, 
                   param_grid=params,
                   scoring='neg_mean_absolute_error', 
                   verbose=1)

clf.fit(X, y)
print("Best parameters:", clf.best_params_)
print("Lowest mae: ", (-clf.best_score_)**(1/2.0))

Fitting 5 folds for each of 54 candidates, totalling 270 fits
Best parameters: {'colsample_bytree': 0.3, 'learning_rate': 0.01, 'max_depth': 6, 'n_estimators': 1000}
Lowest mae:  0.3396954798487488


In [22]:
x_train = bestfeatureslist(x_train, listbestvalues_conscientiousness, 224)
x_test = bestfeatureslist(x_test, listbestvalues_conscientiousness, 224)

In [23]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

data = pd.read_csv("complete_CSV_Train.csv")

X, y = x_train, y_train #1
    
params = { 'max_depth': [3,6,10],
           'learning_rate': [0.01, 0.05, 0.1],
           'n_estimators': [100, 500, 1000],
           'colsample_bytree': [0.3, 0.7]}

xgbr = xgb.XGBRegressor(seed = 20)
clf = GridSearchCV(estimator=xgbr, 
                   param_grid=params,
                   scoring='neg_mean_absolute_error', 
                   verbose=1)

clf.fit(X, y)
print("Best parameters:", clf.best_params_)
print("Lowest mae: ", (-clf.best_score_)**(1/2.0))

Fitting 5 folds for each of 54 candidates, totalling 270 fits
Best parameters: {'colsample_bytree': 0.7, 'learning_rate': 0.01, 'max_depth': 10, 'n_estimators': 1000}
Lowest mae:  0.32791774779364397


In [31]:
csv_file = "complete_CSV_Train.csv"
x_train, y_train = split_data_labels_extroversion(csv_file)
csv_file = "complete_CSV_validation.csv"
x_test, y_test = split_data_labels_extroversion(csv_file)

In [32]:
x_train_10 = bestfeatureslist(x_train, listbestvalues_extroversion, 10)
x_test_10 = bestfeatureslist(x_test, listbestvalues_extroversion, 10)

In [33]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

data = pd.read_csv("complete_CSV_Train.csv")

X, y = x_train_10, y_train #1
    
params = { 'max_depth': [3,6,10],
           'learning_rate': [0.01, 0.05, 0.1],
           'n_estimators': [100, 500, 1000],
           'colsample_bytree': [0.3, 0.7]}

xgbr = xgb.XGBRegressor(seed = 20)
clf = GridSearchCV(estimator=xgbr, 
                   param_grid=params,
                   scoring='neg_mean_absolute_error', 
                   verbose=1)

clf.fit(X, y)
print("Best parameters:", clf.best_params_)
print("Lowest mae: ", (-clf.best_score_)**(1/2.0))

Fitting 5 folds for each of 54 candidates, totalling 270 fits
Best parameters: {'colsample_bytree': 0.7, 'learning_rate': 0.01, 'max_depth': 6, 'n_estimators': 500}
Lowest mae:  0.3260729131610285


In [34]:
x_train_50 = bestfeatureslist(x_train, listbestvalues_extroversion, 50)
x_test_50 = bestfeatureslist(x_test, listbestvalues_extroversion, 50)

In [35]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

data = pd.read_csv("complete_CSV_Train.csv")

X, y = x_train_50, y_train #1
    
params = { 'max_depth': [3,6,10],
           'learning_rate': [0.01, 0.05, 0.1],
           'n_estimators': [100, 500, 1000],
           'colsample_bytree': [0.3, 0.7]}

xgbr = xgb.XGBRegressor(seed = 20)
clf = GridSearchCV(estimator=xgbr, 
                   param_grid=params,
                   scoring='neg_mean_absolute_error', 
                   verbose=1)

clf.fit(X, y)
print("Best parameters:", clf.best_params_)
print("Lowest mae: ", (-clf.best_score_)**(1/2.0))

Fitting 5 folds for each of 54 candidates, totalling 270 fits
Best parameters: {'colsample_bytree': 0.7, 'learning_rate': 0.01, 'max_depth': 6, 'n_estimators': 1000}
Lowest mae:  0.31419569746158105


In [36]:
x_train_100 = bestfeatureslist(x_train, listbestvalues_extroversion, 100)
x_test_100 = bestfeatureslist(x_test, listbestvalues_extroversion, 100)

In [37]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

data = pd.read_csv("complete_CSV_Train.csv")

X, y = x_train_100, y_train #1
    
params = { 'max_depth': [3,6,10],
           'learning_rate': [0.01, 0.05, 0.1],
           'n_estimators': [100, 500, 1000],
           'colsample_bytree': [0.3, 0.7]}

xgbr = xgb.XGBRegressor(seed = 20)
clf = GridSearchCV(estimator=xgbr, 
                   param_grid=params,
                   scoring='neg_mean_absolute_error', 
                   verbose=1)

clf.fit(X, y)
print("Best parameters:", clf.best_params_)
print("Lowest mae: ", (-clf.best_score_)**(1/2.0))

Fitting 5 folds for each of 54 candidates, totalling 270 fits
Best parameters: {'colsample_bytree': 0.3, 'learning_rate': 0.01, 'max_depth': 6, 'n_estimators': 1000}
Lowest mae:  0.31299302463527423


In [38]:
x_train = bestfeatureslist(x_train, listbestvalues_extroversion, 224)
x_test = bestfeatureslist(x_test, listbestvalues_extroversion, 224)

In [39]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

data = pd.read_csv("complete_CSV_Train.csv")

X, y = x_train, y_train #1
    
params = { 'max_depth': [3,6,10],
           'learning_rate': [0.01, 0.05, 0.1],
           'n_estimators': [100, 500, 1000],
           'colsample_bytree': [0.3, 0.7]}

xgbr = xgb.XGBRegressor(seed = 20)
clf = GridSearchCV(estimator=xgbr, 
                   param_grid=params,
                   scoring='neg_mean_absolute_error', 
                   verbose=1)

clf.fit(X, y)
print("Best parameters:", clf.best_params_)
print("Lowest mae: ", (-clf.best_score_)**(1/2.0))

Fitting 5 folds for each of 54 candidates, totalling 270 fits
Best parameters: {'colsample_bytree': 0.7, 'learning_rate': 0.01, 'max_depth': 6, 'n_estimators': 1000}
Lowest mae:  0.31180543951227946


In [40]:
csv_file = "complete_CSV_Train.csv"
x_train, y_train = split_data_labels_agreeableness(csv_file)
csv_file = "complete_CSV_validation.csv"
x_test, y_test = split_data_labels_agreeableness(csv_file)

In [41]:
x_train_10 = bestfeatureslist(x_train, listbestvalues_agreeableness, 10)
x_test_10 = bestfeatureslist(x_test, listbestvalues_agreeableness, 10)

In [42]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

data = pd.read_csv("complete_CSV_Train.csv")

X, y = x_train_10, y_train #1
    
params = { 'max_depth': [3,6,10],
           'learning_rate': [0.01, 0.05, 0.1],
           'n_estimators': [100, 500, 1000],
           'colsample_bytree': [0.3, 0.7]}

xgbr = xgb.XGBRegressor(seed = 20)
clf = GridSearchCV(estimator=xgbr, 
                   param_grid=params,
                   scoring='neg_mean_absolute_error', 
                   verbose=1)

clf.fit(X, y)
print("Best parameters:", clf.best_params_)
print("Lowest mae: ", (-clf.best_score_)**(1/2.0))

Fitting 5 folds for each of 54 candidates, totalling 270 fits
Best parameters: {'colsample_bytree': 0.7, 'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 1000}
Lowest mae:  0.3248431998401827


In [43]:
x_train_50 = bestfeatureslist(x_train, listbestvalues_agreeableness, 50)
x_test_50 = bestfeatureslist(x_test, listbestvalues_agreeableness, 50)

In [44]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

data = pd.read_csv("complete_CSV_Train.csv")

X, y = x_train_50, y_train #1
    
params = { 'max_depth': [3,6,10],
           'learning_rate': [0.01, 0.05, 0.1],
           'n_estimators': [100, 500, 1000],
           'colsample_bytree': [0.3, 0.7]}

xgbr = xgb.XGBRegressor(seed = 20)
clf = GridSearchCV(estimator=xgbr, 
                   param_grid=params,
                   scoring='neg_mean_absolute_error', 
                   verbose=1)

clf.fit(X, y)
print("Best parameters:", clf.best_params_)
print("Lowest mae: ", (-clf.best_score_)**(1/2.0))

Fitting 5 folds for each of 54 candidates, totalling 270 fits
Best parameters: {'colsample_bytree': 0.3, 'learning_rate': 0.01, 'max_depth': 6, 'n_estimators': 1000}
Lowest mae:  0.3207951993246344


In [45]:
x_train_100 = bestfeatureslist(x_train, listbestvalues_agreeableness, 100)
x_test_100 = bestfeatureslist(x_test, listbestvalues_agreeableness, 100)

In [46]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

data = pd.read_csv("complete_CSV_Train.csv")

X, y = x_train_100, y_train #1
    
params = { 'max_depth': [3,6,10],
           'learning_rate': [0.01, 0.05, 0.1],
           'n_estimators': [100, 500, 1000],
           'colsample_bytree': [0.3, 0.7]}

xgbr = xgb.XGBRegressor(seed = 20)
clf = GridSearchCV(estimator=xgbr, 
                   param_grid=params,
                   scoring='neg_mean_absolute_error', 
                   verbose=1)

clf.fit(X, y)
print("Best parameters:", clf.best_params_)
print("Lowest mae: ", (-clf.best_score_)**(1/2.0))

Fitting 5 folds for each of 54 candidates, totalling 270 fits
Best parameters: {'colsample_bytree': 0.7, 'learning_rate': 0.01, 'max_depth': 6, 'n_estimators': 1000}
Lowest mae:  0.31726686933854825


In [47]:
x_train = bestfeatureslist(x_train, listbestvalues_agreeableness, 224)
x_test = bestfeatureslist(x_test, listbestvalues_agreeableness, 224)

In [48]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

data = pd.read_csv("complete_CSV_Train.csv")

X, y = x_train, y_train #1
    
params = { 'max_depth': [3,6,10],
           'learning_rate': [0.01, 0.05, 0.1],
           'n_estimators': [100, 500, 1000],
           'colsample_bytree': [0.3, 0.7]}

xgbr = xgb.XGBRegressor(seed = 20)
clf = GridSearchCV(estimator=xgbr, 
                   param_grid=params,
                   scoring='neg_mean_absolute_error', 
                   verbose=1)

clf.fit(X, y)
print("Best parameters:", clf.best_params_)
print("Lowest mae: ", (-clf.best_score_)**(1/2.0))

Fitting 5 folds for each of 54 candidates, totalling 270 fits
Best parameters: {'colsample_bytree': 0.3, 'learning_rate': 0.01, 'max_depth': 10, 'n_estimators': 1000}
Lowest mae:  0.31518806704480357


In [49]:
csv_file = "complete_CSV_Train.csv"
x_train, y_train = split_data_labels_neuroticism(csv_file)
csv_file = "complete_CSV_validation.csv"
x_test, y_test = split_data_labels_neuroticism(csv_file)

In [50]:
x_train_10 = bestfeatureslist(x_train, listbestvalues_neuroticism, 10)
x_test_10 = bestfeatureslist(x_test, listbestvalues_neuroticism, 10)

In [51]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

data = pd.read_csv("complete_CSV_Train.csv")

X, y = x_train_10, y_train #1
    
params = { 'max_depth': [3,6,10],
           'learning_rate': [0.01, 0.05, 0.1],
           'n_estimators': [100, 500, 1000],
           'colsample_bytree': [0.3, 0.7]}

xgbr = xgb.XGBRegressor(seed = 20)
clf = GridSearchCV(estimator=xgbr, 
                   param_grid=params,
                   scoring='neg_mean_absolute_error', 
                   verbose=1)

clf.fit(X, y)
print("Best parameters:", clf.best_params_)
print("Lowest mae: ", (-clf.best_score_)**(1/2.0))

Fitting 5 folds for each of 54 candidates, totalling 270 fits
Best parameters: {'colsample_bytree': 0.7, 'learning_rate': 0.01, 'max_depth': 6, 'n_estimators': 500}
Lowest mae:  0.33642796923536916


In [52]:
x_train_50 = bestfeatureslist(x_train, listbestvalues_neuroticism, 50)
x_test_50 = bestfeatureslist(x_test, listbestvalues_neuroticism, 50)

In [53]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

data = pd.read_csv("complete_CSV_Train.csv")

X, y = x_train_50, y_train #1
    
params = { 'max_depth': [3,6,10],
           'learning_rate': [0.01, 0.05, 0.1],
           'n_estimators': [100, 500, 1000],
           'colsample_bytree': [0.3, 0.7]}

xgbr = xgb.XGBRegressor(seed = 20)
clf = GridSearchCV(estimator=xgbr, 
                   param_grid=params,
                   scoring='neg_mean_absolute_error', 
                   verbose=1)

clf.fit(X, y)
print("Best parameters:", clf.best_params_)
print("Lowest mae: ", (-clf.best_score_)**(1/2.0))

Fitting 5 folds for each of 54 candidates, totalling 270 fits
Best parameters: {'colsample_bytree': 0.7, 'learning_rate': 0.01, 'max_depth': 10, 'n_estimators': 1000}
Lowest mae:  0.32591546571319985


In [54]:
x_train_100 = bestfeatureslist(x_train, listbestvalues_neuroticism, 100)
x_test_100 = bestfeatureslist(x_test, listbestvalues_neuroticism, 100)

In [55]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

data = pd.read_csv("complete_CSV_Train.csv")

X, y = x_train_100, y_train #1
    
params = { 'max_depth': [3,6,10],
           'learning_rate': [0.01, 0.05, 0.1],
           'n_estimators': [100, 500, 1000],
           'colsample_bytree': [0.3, 0.7]}

xgbr = xgb.XGBRegressor(seed = 20)
clf = GridSearchCV(estimator=xgbr, 
                   param_grid=params,
                   scoring='neg_mean_absolute_error', 
                   verbose=1)

clf.fit(X, y)
print("Best parameters:", clf.best_params_)
print("Lowest mae: ", (-clf.best_score_)**(1/2.0))

Fitting 5 folds for each of 54 candidates, totalling 270 fits
Best parameters: {'colsample_bytree': 0.7, 'learning_rate': 0.01, 'max_depth': 10, 'n_estimators': 1000}
Lowest mae:  0.3242815261093007


In [56]:
x_train = bestfeatureslist(x_train, listbestvalues_neuroticism, 224)
x_test = bestfeatureslist(x_test, listbestvalues_neuroticism, 224)

In [57]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

data = pd.read_csv("complete_CSV_Train.csv")

X, y = x_train, y_train #1
    
params = { 'max_depth': [3,6,10],
           'learning_rate': [0.01, 0.05, 0.1],
           'n_estimators': [100, 500, 1000],
           'colsample_bytree': [0.3, 0.7]}

xgbr = xgb.XGBRegressor(seed = 20)
clf = GridSearchCV(estimator=xgbr, 
                   param_grid=params,
                   scoring='neg_mean_absolute_error', 
                   verbose=1)

clf.fit(X, y)
print("Best parameters:", clf.best_params_)
print("Lowest mae: ", (-clf.best_score_)**(1/2.0))

Fitting 5 folds for each of 54 candidates, totalling 270 fits
Best parameters: {'colsample_bytree': 0.7, 'learning_rate': 0.01, 'max_depth': 10, 'n_estimators': 1000}
Lowest mae:  0.3230100708781289


In [24]:
#XGBoost openness

In [25]:
csv_file = "complete_CSV_Train.csv"
x_train, y_train = split_data_labels_openness(csv_file)
csv_file = "complete_CSV_validation.csv"
x_test, y_test = split_data_labels_openness(csv_file)

In [26]:
x_train_10 = bestfeatureslist(x_train, listbestvalues_openness, 10)
x_test_10 = bestfeatureslist(x_test, listbestvalues_openness, 10)

In [27]:
from xgboost import XGBRegressor

xgbr = XGBRegressor(colsample_bytree = 0.7, learning_rate = 0.01, max_depth = 6, n_estimators = 500, verbosity=0)
print(xgbr)

XGBRegressor(base_score=None, booster=None, colsample_bylevel=None,
             colsample_bynode=None, colsample_bytree=0.7, gamma=None,
             gpu_id=None, importance_type='gain', interaction_constraints=None,
             learning_rate=0.01, max_delta_step=None, max_depth=6,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=500, n_jobs=None, num_parallel_tree=None,
             random_state=None, reg_alpha=None, reg_lambda=None,
             scale_pos_weight=None, subsample=None, tree_method=None,
             validate_parameters=None, verbosity=0)


In [28]:
xgbr.fit(x_train_10, y_train) #1

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.7, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.01, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=500, n_jobs=8, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=0)

In [29]:
score = xgbr.score(x_train_10, y_train) #2
print(score)

0.42525501882508976


In [30]:
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error


cv_score = cross_val_score(xgbr, x_train_10, y_train, cv=10) #3
print("CV mean score:", cv_score.mean())

CV mean score: 0.14169287723162222


In [31]:
ypred = xgbr.predict(x_test_10) #4
mae = mean_absolute_error(y_test, ypred)

print("1-MAE", 1-mae)

1-MAE 0.8944191771580113


In [32]:
x_train_50 = bestfeatureslist(x_train, listbestvalues_openness, 50)
x_test_50 = bestfeatureslist(x_test, listbestvalues_openness, 50)

In [33]:
from xgboost import XGBRegressor

xgbr = XGBRegressor(colsample_bytree = 0.7, learning_rate = 0.01, max_depth = 10, n_estimators = 1000, verbosity=0)
print(xgbr)

XGBRegressor(base_score=None, booster=None, colsample_bylevel=None,
             colsample_bynode=None, colsample_bytree=0.7, gamma=None,
             gpu_id=None, importance_type='gain', interaction_constraints=None,
             learning_rate=0.01, max_delta_step=None, max_depth=10,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=1000, n_jobs=None, num_parallel_tree=None,
             random_state=None, reg_alpha=None, reg_lambda=None,
             scale_pos_weight=None, subsample=None, tree_method=None,
             validate_parameters=None, verbosity=0)


In [34]:
xgbr.fit(x_train_50, y_train) #1

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.7, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.01, max_delta_step=0, max_depth=10,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=1000, n_jobs=8, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=0)

In [35]:
score = xgbr.score(x_train_50, y_train) #2
print(score)

0.9867677654164091


In [37]:
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error


cv_score = cross_val_score(xgbr, x_train_50, y_train, cv=10) #3
print("CV mean score:", cv_score.mean())

CV mean score: 0.23215948773361178


In [38]:
ypred = xgbr.predict(x_test_50) #4
mae = mean_absolute_error(y_test, ypred)

print("1-MAE", 1-mae)

1-MAE 0.9001984692368242


In [39]:
x_train_100 = bestfeatureslist(x_train, listbestvalues_openness, 100)
x_test_100 = bestfeatureslist(x_test, listbestvalues_openness, 100)

In [40]:
from xgboost import XGBRegressor

xgbr = XGBRegressor(colsample_bytree = 0.7, learning_rate = 0.01, max_depth = 10, n_estimators = 1000, verbosity=0)
print(xgbr)

XGBRegressor(base_score=None, booster=None, colsample_bylevel=None,
             colsample_bynode=None, colsample_bytree=0.7, gamma=None,
             gpu_id=None, importance_type='gain', interaction_constraints=None,
             learning_rate=0.01, max_delta_step=None, max_depth=10,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=1000, n_jobs=None, num_parallel_tree=None,
             random_state=None, reg_alpha=None, reg_lambda=None,
             scale_pos_weight=None, subsample=None, tree_method=None,
             validate_parameters=None, verbosity=0)


In [41]:
xgbr.fit(x_train_100, y_train) #1

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.7, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.01, max_delta_step=0, max_depth=10,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=1000, n_jobs=8, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=0)

In [42]:
score = xgbr.score(x_train_100, y_train) #2
print(score)

0.994061110871919


In [43]:
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error


cv_score = cross_val_score(xgbr, x_train_100, y_train, cv=10) #3
print("CV mean score:", cv_score.mean())

CV mean score: 0.25344987886605497


In [44]:
ypred = xgbr.predict(x_test_100) #4
mae = mean_absolute_error(y_test, ypred)

print("1-MAE", 1-mae)

1-MAE 0.9012774353196223


In [45]:
x_train_224 = bestfeatureslist(x_train, listbestvalues_openness, 224)
x_test_224 = bestfeatureslist(x_test, listbestvalues_openness, 224)

In [46]:
from xgboost import XGBRegressor

xgbr = XGBRegressor(colsample_bytree = 0.3, learning_rate = 0.01, max_depth = 6, n_estimators = 1000, verbosity=0)
print(xgbr)

XGBRegressor(base_score=None, booster=None, colsample_bylevel=None,
             colsample_bynode=None, colsample_bytree=0.3, gamma=None,
             gpu_id=None, importance_type='gain', interaction_constraints=None,
             learning_rate=0.01, max_delta_step=None, max_depth=6,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=1000, n_jobs=None, num_parallel_tree=None,
             random_state=None, reg_alpha=None, reg_lambda=None,
             scale_pos_weight=None, subsample=None, tree_method=None,
             validate_parameters=None, verbosity=0)


In [47]:
xgbr.fit(x_train_224, y_train) #1

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.3, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.01, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=1000, n_jobs=8, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=0)

In [48]:
score = xgbr.score(x_train_224, y_train) #2
print(score)

0.7917210723322067


In [49]:
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error


cv_score = cross_val_score(xgbr, x_train_224, y_train, cv=10) #3
print("CV mean score:", cv_score.mean())

CV mean score: 0.2756652366241622


In [50]:
ypred = xgbr.predict(x_test_224) #4
mae = mean_absolute_error(y_test, ypred)

print("1-MAE", 1-mae)

1-MAE 0.9034726799226469


In [ ]:
#XGBoost conscientiousness

In [51]:
csv_file = "complete_CSV_Train.csv"
x_train, y_train = split_data_labels_conscientiousness(csv_file)
csv_file = "complete_CSV_validation.csv"
x_test, y_test = split_data_labels_conscientiousness(csv_file)

In [52]:
x_train_10 = bestfeatureslist(x_train, listbestvalues_conscientiousness, 10)
x_test_10 = bestfeatureslist(x_test, listbestvalues_conscientiousness, 10)

In [53]:
from xgboost import XGBRegressor

xgbr = XGBRegressor(colsample_bytree = 0.3, learning_rate = 0.01, max_depth = 3, n_estimators = 1000, verbosity=0)
print(xgbr)

XGBRegressor(base_score=None, booster=None, colsample_bylevel=None,
             colsample_bynode=None, colsample_bytree=0.3, gamma=None,
             gpu_id=None, importance_type='gain', interaction_constraints=None,
             learning_rate=0.01, max_delta_step=None, max_depth=3,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=1000, n_jobs=None, num_parallel_tree=None,
             random_state=None, reg_alpha=None, reg_lambda=None,
             scale_pos_weight=None, subsample=None, tree_method=None,
             validate_parameters=None, verbosity=0)


In [54]:
xgbr.fit(x_train_10, y_train) #1

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.3, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.01, max_delta_step=0, max_depth=3,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=1000, n_jobs=8, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=0)

In [55]:
score = xgbr.score(x_train_10, y_train) #2
print(score)

0.15655031231860939


In [56]:
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error


cv_score = cross_val_score(xgbr, x_train_10, y_train, cv=10) #3
print("CV mean score:", cv_score.mean())

CV mean score: 0.07006297326839583


In [57]:
ypred = xgbr.predict(x_test_10) #4
mae = mean_absolute_error(y_test, ypred)

print("1-MAE", 1-mae)

1-MAE 0.8793791132138192


In [64]:
x_train_50 = bestfeatureslist(x_train, listbestvalues_conscientiousness, 50)
x_test_50 = bestfeatureslist(x_test, listbestvalues_conscientiousness, 50)

In [65]:
from xgboost import XGBRegressor

xgbr = XGBRegressor(colsample_bytree = 0.7, learning_rate = 0.01, max_depth = 6, n_estimators = 500, verbosity=0)
print(xgbr)

XGBRegressor(base_score=None, booster=None, colsample_bylevel=None,
             colsample_bynode=None, colsample_bytree=0.7, gamma=None,
             gpu_id=None, importance_type='gain', interaction_constraints=None,
             learning_rate=0.01, max_delta_step=None, max_depth=6,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=500, n_jobs=None, num_parallel_tree=None,
             random_state=None, reg_alpha=None, reg_lambda=None,
             scale_pos_weight=None, subsample=None, tree_method=None,
             validate_parameters=None, verbosity=0)


In [66]:
xgbr.fit(x_train_50, y_train) #1

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.7, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.01, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=500, n_jobs=8, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=0)

In [67]:
score = xgbr.score(x_train_50, y_train) #2
print(score)

0.4252550459952389


In [68]:
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error


cv_score = cross_val_score(xgbr, x_train_50, y_train, cv=10) #3
print("CV mean score:", cv_score.mean())

CV mean score: 0.10470034694081809


In [69]:
ypred = xgbr.predict(x_test_50) #4
mae = mean_absolute_error(y_test, ypred)

print("1-MAE", 1-mae)

1-MAE 0.8835694304039177


In [70]:
x_train_100 = bestfeatureslist(x_train, listbestvalues_conscientiousness, 100)
x_test_100 = bestfeatureslist(x_test, listbestvalues_conscientiousness, 100)

In [71]:
from xgboost import XGBRegressor

xgbr = XGBRegressor(colsample_bytree = 0.3, learning_rate = 0.01, max_depth = 6, n_estimators = 1000, verbosity=0)
print(xgbr)

XGBRegressor(base_score=None, booster=None, colsample_bylevel=None,
             colsample_bynode=None, colsample_bytree=0.3, gamma=None,
             gpu_id=None, importance_type='gain', interaction_constraints=None,
             learning_rate=0.01, max_delta_step=None, max_depth=6,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=1000, n_jobs=None, num_parallel_tree=None,
             random_state=None, reg_alpha=None, reg_lambda=None,
             scale_pos_weight=None, subsample=None, tree_method=None,
             validate_parameters=None, verbosity=0)


In [72]:
xgbr.fit(x_train_100, y_train) #1

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.3, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.01, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=1000, n_jobs=8, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=0)

In [73]:
score = xgbr.score(x_train_100, y_train) #2
print(score)

0.6447879082459275


In [74]:
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error


cv_score = cross_val_score(xgbr, x_train_100, y_train, cv=10) #3
print("CV mean score:", cv_score.mean())

CV mean score: 0.15428568453596841


In [75]:
ypred = xgbr.predict(x_test_100) #4
mae = mean_absolute_error(y_test, ypred)

print("1-MAE", 1-mae)

1-MAE 0.8870086063683322


In [76]:
x_train_224 = bestfeatureslist(x_train, listbestvalues_conscientiousness, 224)
x_test_224 = bestfeatureslist(x_test, listbestvalues_conscientiousness, 224)

In [77]:
from xgboost import XGBRegressor

xgbr = XGBRegressor(colsample_bytree = 0.7, learning_rate = 0.01, max_depth = 10, n_estimators = 1000, verbosity=0)
print(xgbr)

XGBRegressor(base_score=None, booster=None, colsample_bylevel=None,
             colsample_bynode=None, colsample_bytree=0.7, gamma=None,
             gpu_id=None, importance_type='gain', interaction_constraints=None,
             learning_rate=0.01, max_delta_step=None, max_depth=10,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=1000, n_jobs=None, num_parallel_tree=None,
             random_state=None, reg_alpha=None, reg_lambda=None,
             scale_pos_weight=None, subsample=None, tree_method=None,
             validate_parameters=None, verbosity=0)


In [78]:
xgbr.fit(x_train_224, y_train) #1

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.7, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.01, max_delta_step=0, max_depth=10,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=1000, n_jobs=8, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=0)

In [79]:
score = xgbr.score(x_train_224, y_train) #2
print(score)

0.9941307191124407


In [80]:
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error


cv_score = cross_val_score(xgbr, x_train_224, y_train, cv=10) #3
print("CV mean score:", cv_score.mean())

CV mean score: 0.2645229263778699


In [81]:
ypred = xgbr.predict(x_test_224) #4
mae = mean_absolute_error(y_test, ypred)

print("1-MAE", 1-mae)

1-MAE 0.8959409390377622


In [ ]:
#XGBoost extroversion

In [82]:
csv_file = "complete_CSV_Train.csv"
x_train, y_train = split_data_labels_extroversion(csv_file)
csv_file = "complete_CSV_validation.csv"
x_test, y_test = split_data_labels_extroversion(csv_file)

In [83]:
x_train_10 = bestfeatureslist(x_train, listbestvalues_extroversion, 10)
x_test_10 = bestfeatureslist(x_test, listbestvalues_extroversion, 10)

In [84]:
from xgboost import XGBRegressor

xgbr = XGBRegressor(colsample_bytree = 0.7, learning_rate = 0.01, max_depth = 6, n_estimators = 500, verbosity=0)
print(xgbr)

XGBRegressor(base_score=None, booster=None, colsample_bylevel=None,
             colsample_bynode=None, colsample_bytree=0.7, gamma=None,
             gpu_id=None, importance_type='gain', interaction_constraints=None,
             learning_rate=0.01, max_delta_step=None, max_depth=6,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=500, n_jobs=None, num_parallel_tree=None,
             random_state=None, reg_alpha=None, reg_lambda=None,
             scale_pos_weight=None, subsample=None, tree_method=None,
             validate_parameters=None, verbosity=0)


In [85]:
xgbr.fit(x_train_10, y_train) #1

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.7, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.01, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=500, n_jobs=8, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=0)

In [86]:
score = xgbr.score(x_train_10, y_train) #2
print(score)

0.4989008898873364


In [87]:
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error


cv_score = cross_val_score(xgbr, x_train_10, y_train, cv=10) #3
print("CV mean score:", cv_score.mean())

CV mean score: 0.22937561118128796


In [88]:
ypred = xgbr.predict(x_test_10) #4
mae = mean_absolute_error(y_test, ypred)

print("1-MAE", 1-mae)

1-MAE 0.896627368362429


In [89]:
x_train_50 = bestfeatureslist(x_train, listbestvalues_extroversion, 50)
x_test_50 = bestfeatureslist(x_test, listbestvalues_extroversion, 50)

In [90]:
from xgboost import XGBRegressor

xgbr = XGBRegressor(colsample_bytree = 0.7, learning_rate = 0.01, max_depth = 6, n_estimators = 1000, verbosity=0)
print(xgbr)

XGBRegressor(base_score=None, booster=None, colsample_bylevel=None,
             colsample_bynode=None, colsample_bytree=0.7, gamma=None,
             gpu_id=None, importance_type='gain', interaction_constraints=None,
             learning_rate=0.01, max_delta_step=None, max_depth=6,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=1000, n_jobs=None, num_parallel_tree=None,
             random_state=None, reg_alpha=None, reg_lambda=None,
             scale_pos_weight=None, subsample=None, tree_method=None,
             validate_parameters=None, verbosity=0)


In [91]:
xgbr.fit(x_train_50, y_train) #1

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.7, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.01, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=1000, n_jobs=8, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=0)

In [92]:
score = xgbr.score(x_train_50, y_train) #2
print(score)

0.753452737354478


In [93]:
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error


cv_score = cross_val_score(xgbr, x_train_50, y_train, cv=10) #3
print("CV mean score:", cv_score.mean())

CV mean score: 0.33390524707361735


In [94]:
ypred = xgbr.predict(x_test_50) #4
mae = mean_absolute_error(y_test, ypred)

print("1-MAE", 1-mae)

1-MAE 0.9043758539732232


In [95]:
x_train_100 = bestfeatureslist(x_train, listbestvalues_extroversion, 100)
x_test_100 = bestfeatureslist(x_test, listbestvalues_extroversion, 100)

In [96]:
from xgboost import XGBRegressor

xgbr = XGBRegressor(colsample_bytree = 0.3, learning_rate = 0.01, max_depth = 6, n_estimators = 1000, verbosity=0)
print(xgbr)

XGBRegressor(base_score=None, booster=None, colsample_bylevel=None,
             colsample_bynode=None, colsample_bytree=0.3, gamma=None,
             gpu_id=None, importance_type='gain', interaction_constraints=None,
             learning_rate=0.01, max_delta_step=None, max_depth=6,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=1000, n_jobs=None, num_parallel_tree=None,
             random_state=None, reg_alpha=None, reg_lambda=None,
             scale_pos_weight=None, subsample=None, tree_method=None,
             validate_parameters=None, verbosity=0)


In [97]:
xgbr.fit(x_train_100, y_train) #1

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.3, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.01, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=1000, n_jobs=8, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=0)

In [98]:
score = xgbr.score(x_train_100, y_train) #2
print(score)

0.7764749727084175


In [99]:
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error


cv_score = cross_val_score(xgbr, x_train_100, y_train, cv=10) #3
print("CV mean score:", cv_score.mean())

CV mean score: 0.3433772175105627


In [100]:
ypred = xgbr.predict(x_test_100) #4
mae = mean_absolute_error(y_test, ypred)

print("1-MAE", 1-mae)

1-MAE 0.9055461702578674


In [101]:
x_train_224 = bestfeatureslist(x_train, listbestvalues_extroversion, 224)
x_test_224 = bestfeatureslist(x_test, listbestvalues_extroversion, 224)

In [102]:
from xgboost import XGBRegressor

xgbr = XGBRegressor(colsample_bytree = 0.7, learning_rate = 0.01, max_depth = 6, n_estimators = 1000, verbosity=0)
print(xgbr)

XGBRegressor(base_score=None, booster=None, colsample_bylevel=None,
             colsample_bynode=None, colsample_bytree=0.7, gamma=None,
             gpu_id=None, importance_type='gain', interaction_constraints=None,
             learning_rate=0.01, max_delta_step=None, max_depth=6,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=1000, n_jobs=None, num_parallel_tree=None,
             random_state=None, reg_alpha=None, reg_lambda=None,
             scale_pos_weight=None, subsample=None, tree_method=None,
             validate_parameters=None, verbosity=0)


In [103]:
xgbr.fit(x_train_224, y_train) #1

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.7, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.01, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=1000, n_jobs=8, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=0)

In [104]:
score = xgbr.score(x_train_224, y_train) #2
print(score)

0.8184858919539533


In [105]:
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error


cv_score = cross_val_score(xgbr, x_train_224, y_train, cv=10) #3
print("CV mean score:", cv_score.mean())

CV mean score: 0.355500367565168


In [106]:
ypred = xgbr.predict(x_test_224) #4
mae = mean_absolute_error(y_test, ypred)

print("1-MAE", 1-mae)

1-MAE 0.9062166214810214


In [ ]:
#XGBoost agreeableness

In [107]:
csv_file = "complete_CSV_Train.csv"
x_train, y_train = split_data_labels_agreeableness(csv_file)
csv_file = "complete_CSV_validation.csv"
x_test, y_test = split_data_labels_agreeableness(csv_file)

In [108]:
x_train_10 = bestfeatureslist(x_train, listbestvalues_agreeableness, 10)
x_test_10 = bestfeatureslist(x_test, listbestvalues_agreeableness, 10)

In [109]:
from xgboost import XGBRegressor

xgbr = XGBRegressor(colsample_bytree = 0.7, learning_rate = 0.01, max_depth = 3, n_estimators = 1000, verbosity=0)
print(xgbr)

XGBRegressor(base_score=None, booster=None, colsample_bylevel=None,
             colsample_bynode=None, colsample_bytree=0.7, gamma=None,
             gpu_id=None, importance_type='gain', interaction_constraints=None,
             learning_rate=0.01, max_delta_step=None, max_depth=3,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=1000, n_jobs=None, num_parallel_tree=None,
             random_state=None, reg_alpha=None, reg_lambda=None,
             scale_pos_weight=None, subsample=None, tree_method=None,
             validate_parameters=None, verbosity=0)


In [110]:
xgbr.fit(x_train_10, y_train) #1

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.7, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.01, max_delta_step=0, max_depth=3,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=1000, n_jobs=8, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=0)

In [111]:
score = xgbr.score(x_train_10, y_train) #2
print(score)

0.14344023040289866


In [112]:
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error


cv_score = cross_val_score(xgbr, x_train_10, y_train, cv=10) #3
print("CV mean score:", cv_score.mean())

CV mean score: 0.04517301317963851


In [113]:
ypred = xgbr.predict(x_test_10) #4
mae = mean_absolute_error(y_test, ypred)

print("1-MAE", 1-mae)

1-MAE 0.9024170515828736


In [114]:
x_train_50 = bestfeatureslist(x_train, listbestvalues_agreeableness, 50)
x_test_50 = bestfeatureslist(x_test, listbestvalues_agreeableness, 50)

In [115]:
from xgboost import XGBRegressor

xgbr = XGBRegressor(colsample_bytree = 0.3, learning_rate = 0.01, max_depth = 6, n_estimators = 1000, verbosity=0)
print(xgbr)

XGBRegressor(base_score=None, booster=None, colsample_bylevel=None,
             colsample_bynode=None, colsample_bytree=0.3, gamma=None,
             gpu_id=None, importance_type='gain', interaction_constraints=None,
             learning_rate=0.01, max_delta_step=None, max_depth=6,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=1000, n_jobs=None, num_parallel_tree=None,
             random_state=None, reg_alpha=None, reg_lambda=None,
             scale_pos_weight=None, subsample=None, tree_method=None,
             validate_parameters=None, verbosity=0)


In [116]:
xgbr.fit(x_train_50, y_train) #1

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.3, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.01, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=1000, n_jobs=8, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=0)

In [117]:
score = xgbr.score(x_train_50, y_train) #2
print(score)

0.5463352206012


In [118]:
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error


cv_score = cross_val_score(xgbr, x_train_50, y_train, cv=10) #3
print("CV mean score:", cv_score.mean())

CV mean score: 0.09189522882766019


In [119]:
ypred = xgbr.predict(x_test_50) #4
mae = mean_absolute_error(y_test, ypred)

print("1-MAE", 1-mae)

1-MAE 0.9042304898002645


In [120]:
x_train_100 = bestfeatureslist(x_train, listbestvalues_agreeableness, 100)
x_test_100 = bestfeatureslist(x_test, listbestvalues_agreeableness, 100)

In [121]:
from xgboost import XGBRegressor

xgbr = XGBRegressor(colsample_bytree = 0.7, learning_rate = 0.01, max_depth = 6, n_estimators = 1000, verbosity=0)
print(xgbr)

XGBRegressor(base_score=None, booster=None, colsample_bylevel=None,
             colsample_bynode=None, colsample_bytree=0.7, gamma=None,
             gpu_id=None, importance_type='gain', interaction_constraints=None,
             learning_rate=0.01, max_delta_step=None, max_depth=6,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=1000, n_jobs=None, num_parallel_tree=None,
             random_state=None, reg_alpha=None, reg_lambda=None,
             scale_pos_weight=None, subsample=None, tree_method=None,
             validate_parameters=None, verbosity=0)


In [122]:
xgbr.fit(x_train_100, y_train) #1

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.7, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.01, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=1000, n_jobs=8, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=0)

In [123]:
score = xgbr.score(x_train_100, y_train) #2
print(score)

0.6515535647404972


In [124]:
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error


cv_score = cross_val_score(xgbr, x_train_100, y_train, cv=10) #3
print("CV mean score:", cv_score.mean())

CV mean score: 0.13311733632252426


In [125]:
ypred = xgbr.predict(x_test_100) #4
mae = mean_absolute_error(y_test, ypred)

print("1-MAE", 1-mae)

1-MAE 0.9058317683565092


In [126]:
x_train_224 = bestfeatureslist(x_train, listbestvalues_agreeableness, 224)
x_test_224 = bestfeatureslist(x_test, listbestvalues_agreeableness, 224)

In [127]:
from xgboost import XGBRegressor

xgbr = XGBRegressor(colsample_bytree = 0.3, learning_rate = 0.01, max_depth = 10, n_estimators = 1000, verbosity=0)
print(xgbr)

XGBRegressor(base_score=None, booster=None, colsample_bylevel=None,
             colsample_bynode=None, colsample_bytree=0.3, gamma=None,
             gpu_id=None, importance_type='gain', interaction_constraints=None,
             learning_rate=0.01, max_delta_step=None, max_depth=10,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=1000, n_jobs=None, num_parallel_tree=None,
             random_state=None, reg_alpha=None, reg_lambda=None,
             scale_pos_weight=None, subsample=None, tree_method=None,
             validate_parameters=None, verbosity=0)


In [128]:
xgbr.fit(x_train_224, y_train) #1

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.3, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.01, max_delta_step=0, max_depth=10,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=1000, n_jobs=8, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=0)

In [129]:
score = xgbr.score(x_train_224, y_train) #2
print(score)

0.9906076694308384


In [130]:
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error


cv_score = cross_val_score(xgbr, x_train_224, y_train, cv=10) #3
print("CV mean score:", cv_score.mean())

CV mean score: 0.15970091364667596


In [131]:
ypred = xgbr.predict(x_test_224) #4
mae = mean_absolute_error(y_test, ypred)

print("1-MAE", 1-mae)

1-MAE 0.9081222003446175


In [ ]:
#XGBoost neuroticism

In [132]:
csv_file = "complete_CSV_Train.csv"
x_train, y_train = split_data_labels_neuroticism(csv_file)
csv_file = "complete_CSV_validation.csv"
x_test, y_test = split_data_labels_neuroticism(csv_file)

In [133]:
x_train_10 = bestfeatureslist(x_train, listbestvalues_neuroticism, 10)
x_test_10 = bestfeatureslist(x_test, listbestvalues_neuroticism, 10)

In [134]:
from xgboost import XGBRegressor

xgbr = XGBRegressor(colsample_bytree = 0.7, learning_rate = 0.01, max_depth = 3, n_estimators = 1000, verbosity=0)
print(xgbr)

XGBRegressor(base_score=None, booster=None, colsample_bylevel=None,
             colsample_bynode=None, colsample_bytree=0.7, gamma=None,
             gpu_id=None, importance_type='gain', interaction_constraints=None,
             learning_rate=0.01, max_delta_step=None, max_depth=3,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=1000, n_jobs=None, num_parallel_tree=None,
             random_state=None, reg_alpha=None, reg_lambda=None,
             scale_pos_weight=None, subsample=None, tree_method=None,
             validate_parameters=None, verbosity=0)


In [135]:
xgbr.fit(x_train_10, y_train) #1

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.7, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.01, max_delta_step=0, max_depth=3,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=1000, n_jobs=8, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=0)

In [136]:
score = xgbr.score(x_train_10, y_train) #2
print(score)

0.25518737409234427


In [137]:
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error


cv_score = cross_val_score(xgbr, x_train_10, y_train, cv=10) #3
print("CV mean score:", cv_score.mean())

CV mean score: 0.14822913743015617


In [138]:
ypred = xgbr.predict(x_test_10) #4
mae = mean_absolute_error(y_test, ypred)

print("1-MAE", 1-mae)

1-MAE 0.8884211068501076


In [139]:
x_train_50 = bestfeatureslist(x_train, listbestvalues_neuroticism, 50)
x_test_50 = bestfeatureslist(x_test, listbestvalues_neuroticism, 50)

In [140]:
from xgboost import XGBRegressor

xgbr = XGBRegressor(colsample_bytree = 0.7, learning_rate = 0.01, max_depth = 10, n_estimators = 1000, verbosity=0)
print(xgbr)

XGBRegressor(base_score=None, booster=None, colsample_bylevel=None,
             colsample_bynode=None, colsample_bytree=0.7, gamma=None,
             gpu_id=None, importance_type='gain', interaction_constraints=None,
             learning_rate=0.01, max_delta_step=None, max_depth=10,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=1000, n_jobs=None, num_parallel_tree=None,
             random_state=None, reg_alpha=None, reg_lambda=None,
             scale_pos_weight=None, subsample=None, tree_method=None,
             validate_parameters=None, verbosity=0)


In [141]:
xgbr.fit(x_train_50, y_train) #1

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.7, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.01, max_delta_step=0, max_depth=10,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=1000, n_jobs=8, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=0)

In [142]:
score = xgbr.score(x_train_50, y_train) #2
print(score)

0.9875920819766646


In [144]:
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error


cv_score = cross_val_score(xgbr, x_train_50, y_train, cv=10) #3
print("CV mean score:", cv_score.mean())

CV mean score: 0.2419727034451455


In [145]:
ypred = xgbr.predict(x_test_50) #4
mae = mean_absolute_error(y_test, ypred)

print("1-MAE", 1-mae)

1-MAE 0.8959150603537758


In [146]:
x_train_100 = bestfeatureslist(x_train, listbestvalues_neuroticism, 100)
x_test_100 = bestfeatureslist(x_test, listbestvalues_neuroticism, 100)

In [147]:
from xgboost import XGBRegressor

xgbr = XGBRegressor(colsample_bytree = 0.7, learning_rate = 0.01, max_depth = 10, n_estimators = 1000, verbosity=0)
print(xgbr)

XGBRegressor(base_score=None, booster=None, colsample_bylevel=None,
             colsample_bynode=None, colsample_bytree=0.7, gamma=None,
             gpu_id=None, importance_type='gain', interaction_constraints=None,
             learning_rate=0.01, max_delta_step=None, max_depth=10,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=1000, n_jobs=None, num_parallel_tree=None,
             random_state=None, reg_alpha=None, reg_lambda=None,
             scale_pos_weight=None, subsample=None, tree_method=None,
             validate_parameters=None, verbosity=0)


In [148]:
xgbr.fit(x_train_100, y_train) #1

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.7, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.01, max_delta_step=0, max_depth=10,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=1000, n_jobs=8, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=0)

In [149]:
score = xgbr.score(x_train_100, y_train) #2
print(score)

0.9927657505724209


In [150]:
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error


cv_score = cross_val_score(xgbr, x_train_100, y_train, cv=10) #3
print("CV mean score:", cv_score.mean())

CV mean score: 0.2596123933550749


In [151]:
ypred = xgbr.predict(x_test_100) #4
mae = mean_absolute_error(y_test, ypred)

print("1-MAE", 1-mae)

1-MAE 0.897210583627224


In [152]:
x_train_224 = bestfeatureslist(x_train, listbestvalues_neuroticism, 224)
x_test_224 = bestfeatureslist(x_test, listbestvalues_neuroticism, 224)

In [153]:
from xgboost import XGBRegressor

xgbr = XGBRegressor(colsample_bytree = 0.7, learning_rate = 0.01, max_depth = 10, n_estimators = 1000, verbosity=0)
print(xgbr)

XGBRegressor(base_score=None, booster=None, colsample_bylevel=None,
             colsample_bynode=None, colsample_bytree=0.7, gamma=None,
             gpu_id=None, importance_type='gain', interaction_constraints=None,
             learning_rate=0.01, max_delta_step=None, max_depth=10,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=1000, n_jobs=None, num_parallel_tree=None,
             random_state=None, reg_alpha=None, reg_lambda=None,
             scale_pos_weight=None, subsample=None, tree_method=None,
             validate_parameters=None, verbosity=0)


In [154]:
xgbr.fit(x_train_224, y_train) #1

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.7, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.01, max_delta_step=0, max_depth=10,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=1000, n_jobs=8, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=0)

In [155]:
score = xgbr.score(x_train_224, y_train) #2
print(score)

0.9958282184516131


In [156]:
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error


cv_score = cross_val_score(xgbr, x_train_224, y_train, cv=10) #3
print("CV mean score:", cv_score.mean())

CV mean score: 0.2690112299347122


In [157]:
ypred = xgbr.predict(x_test_224) #4
mae = mean_absolute_error(y_test, ypred)

print("1-MAE", 1-mae)

1-MAE 0.8983270758713285
